In [ ]:
# This cell is added by sphinx-gallery
!pip install mrsimulator --quiet


%matplotlib inline

import mrsimulator
print(f'You are using mrsimulator v{mrsimulator.__version__}')


# Arbitrary spin transition (single-quantum)

²⁷Al (I=5/2) quadrupolar spectrum simulation.


The mrsimulator built-in one-dimensional methods, BlochDecaySpectrum and
BlochDecayCTSpectrum, are designed to simulate spectrum from all single
quantum transitions or central transition selective transition, respectively. In this
example, we show how you can simulate any arbitrary transition using the generic
Method1D method.



In [ ]:
import matplotlib.pyplot as plt

from mrsimulator import Simulator, SpinSystem, Site
from mrsimulator.methods import Method1D

Create a single-site arbitrary spin system.



In [ ]:
site = Site(
    name="27Al",
    isotope="27Al",
    isotropic_chemical_shift=35.7,  # in ppm
    quadrupolar={"Cq": 5.959e6, "eta": 0.32},  # Cq is in Hz
)
spin_system = SpinSystem(sites=[site])

## Selecting spin transitions for simulation

The arguments of the Method1D object are the same as the arguments of the
BlochDecaySpectrum method; however, unlike a BlochDecaySpectrum method, the
`spectral_dim_api` object in Method1D contains additional argument---`events`.

The `event_api` object is a collection of attributes, which are local to the
event. It is here where we define a `transition_query` to select one or more
transitions for simulating the spectrum. The two attributes of the `transition_query`
are `P` and `D`, which are given as,

\begin{align}P &= m_f - m_i \\
    D &= m_f^2 - m_i^2,\end{align}

where $m_f$ and $m_i$ are the spin quantum numbers for the final and
initial energy states. Based on the query, the method selects all transitions from
the spin system that satisfy the query selection criterion. For example, to simulate
a spectrum for the satellite transition, $|-1/2\rangle\rightarrow|-3/2\rangle$,
set the value of

\begin{align}P &= \left(-\frac{3}{2}\right) - \left(-\frac{1}{2}\right) = -1 \\
    D &= \frac{9}{4} - \frac{1}{4} = 2.\end{align}

For illustrative purposes, let's look at the infinite speed spectrum from this
satellite transition.



In [ ]:
method = Method1D(
    name="Inner Satellite Spectrum",
    channels=["27Al"],
    magnetic_flux_density=21.14,  # in T
    rotor_frequency=1e9,  # in Hz
    spectral_dimensions=[
        {
            "count": 1024,
            "spectral_width": 1e4,  # in Hz
            "reference_offset": 1e4,  # in Hz
            "events": [
                {
                    "transition_query": [
                        {"P": [-1], "D": [2]}  # <-- select inner satellite transitions
                    ]
                }
            ],
        }
    ],
)

# A graphical representation of the method object.
plt.figure(figsize=(5, 3))
method.plot()
plt.show()

Create the Simulator object and add the method and the spin system object.



In [ ]:
sim = Simulator()
sim.spin_systems += [spin_system]  # add the spin system
sim.methods += [method]  # add the method

Simulate the spectrum.



In [ ]:
sim.run()

# The plot of the simulation before signal processing.
plt.figure(figsize=(4.25, 3.0))
ax = plt.subplot(projection="csdm")
ax.plot(sim.methods[0].simulation.real, color="black", linewidth=1)
ax.invert_xaxis()
plt.tight_layout()
plt.show()

## Selecting both inner and outer-satellite transitions
You may use the same transition query selection criterion to select multiple
transitions. Consider the following transitions with respective P and D values.

- $|-1/2\rangle\rightarrow|-3/2\rangle$ ($P=-1, D=2$)
- $|-3/2\rangle\rightarrow|-5/2\rangle$ ($P=-1, D=4$)



In [ ]:
method2 = Method1D(
    name="Satellite Spectrum",
    channels=["27Al"],
    magnetic_flux_density=21.14,  # in T
    rotor_frequency=1e9,  # in Hz
    spectral_dimensions=[
        {
            "count": 1024,
            "spectral_width": 1e4,  # in Hz
            "reference_offset": 1e4,  # in Hz
            "events": [
                {
                    "transition_query": [
                        {"P": [-1], "D": [2]},  # <-- select inter satellite transitions
                        {"P": [-1], "D": [4]},  # <-- select outer satellite transitions
                    ]
                }
            ],
        }
    ],
)

# A graphical representation of the method object.
plt.figure(figsize=(5, 3))
method2.plot()
plt.show()

Update the method object in the Simulator object.



In [ ]:
sim.methods[0] = method2  # add the method

Simulate the spectrum.



In [ ]:
sim.run()

# The plot of the simulation before signal processing.
plt.figure(figsize=(4.25, 3.0))
ax = plt.subplot(projection="csdm")
ax.plot(sim.methods[0].simulation.real, color="black", linewidth=1)
ax.invert_xaxis()
plt.tight_layout()
plt.show()